# 올드한 모리 코고로 탐정과 대화하기 (LCEL)

체인 파이프라인(LCEL)을 사용.

## OpenAI LLM 준비 및 퀴즈 파일 지정
* 환경 변수(`.env` 파일)에서 API Key 로딩
* 개발 환경에서는 `gpt-4o-mini` 또는 `gpt-3.5-turbo`

In [ ]:
import gradio as gr
import random

from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.agents import AgentType, initialize_agent
from langchain.memory import ConversationBufferMemory

from langchain.prompts.chat import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


load_dotenv()

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)

## 시스템 메시지 (페르소나) 지정

In [ ]:

# 1. 모리 코고로 페르소나 정의
persona = """
당신은 애니메이션 '명탐정 코난'의 등장인물 '모리 코고로'입니다. 
당신은 자칭 명탐정이며, 겉으로는 허세와 자신감이 넘치지만 사실은 허당에 가까운 면모도 있는 인물입니다. 
여성에게 약하고, 술과 도박을 즐기며, 평소에는 게으르지만 가족이 위험에 처하면 누구보다 진심 어린 모습을 보입니다. 
당신은 경찰 출신으로 유도와 사격 실력이 뛰어나며, 과거의 명성을 은근히 자랑하곤 합니다.

말투는 다소 거칠고 직설적이며, 자신감 넘치는 표현을 자주 사용합니다. 
예를 들어, "이 몸이 바로 명탐정 모리 코고로다!", "흠… 이건 분명히 범인의 소행이군!", "핫핫하, 그런 건 내게 맡기라고!" 와 같은 말을 즐겨 사용합니다.
추리 상황에서는 진지하면서도 과장된 표현을 즐깁니다.

당신은 지금부터 어떤 질문에도 모리 코고로답게 답변해야 하며, 질문자에게 당신이 진짜 명탐정이라는 인상을 주기 위해 노력합니다. 
가끔 "그 꼬맹이는 또 어디서 뭘 하고 있는 거야?"라며 코난에게 얄밉게 툴툴대는 듯한 멘트도 허용됩니다.

당신은 지금부터 '모리 코고로' 그 자체입니다. 답변의 주요한 단어 뒤에는 적절한 유니코드 이모지를 추가합니다.
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", persona),
        ("human", "{question}"),
    ]
)

chain = prompt | llm | StrOutputParser()


## UI 인터페이스

In [ ]:
mori_fillers = [
    "😎 그건 말이지... 나 모리 코고로가 보기엔 말이야. 🍷",
    "🧐 흠… 잘 들어봐. 중요한 건 말이지… 🔍",
    "😏 에헴, 그러니까 말이지… 이건 좀 수상하단 말이야. 💭",
    "💼 그야 당연하지! 하지만 말이야, 진실은 항상 하나거든. 🎯",
    "🕵️‍♂️ 후후, 이제야 감이 오는군. 그건 말이야… 💡"
]

def play_caht(message, history):
    history.append({"role": "user", "content": message})
    history.append({"role": "assistant", "content": random.choice(mori_fillers)})
    yield history

    chain_response = chain.invoke({"question": message})
    history[-1]["content"] = chain_response
    yield history


## UI 채팅 화면

In [ ]:
# Gradio UI 레이아웃
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("## 명탐정 모리 코고로 🕵️‍♂️")
    gr.Markdown("이 몸이 바로 잠자는 코고로, 명탐정 모리 코고로다! 무슨 사건이든 물어보라고. 핫핫하!")

    chatbot = gr.Chatbot(label="대화창", height=300, type="messages")
    user_input = gr.Textbox(placeholder="무엇이든 물어보라구...음핫핫핫!", show_label=False)

    # 입력 예시를 examples로 설정
    gr.Examples(
        examples=[["모리 아저씨, 당신도 명탐정인가요?"], ["최근에 해결한 사건에 대해 말해 주세요."], ["코난은 어디에 있어요?"]],
        inputs=user_input,
        cache_examples=False
    )

    user_input.submit(
        fn=play_caht,
        inputs=[user_input, chatbot],
        outputs=[chatbot]
    )

    user_input.submit(lambda: "", None, user_input)

# 웹 서버 실행
demo.launch()

In [ ]:
demo.close()